In [2]:
import pandas as pd
import streamlit as st

df = pd.read_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/df.xlsx')
df.head()

,Unnamed: 0,A,B,C,D,E,F,G,H,I,...,R,CS,PAL,OUT,Picks PAL,Picks CS,Picks OUT,Picks Gesamt,Kalender Woche,Monat
0,0,3200817575,2022-06-21,2022-06-22,5272903180,ZLF,NaN,DE30,DE30,50034469,...,C,4.25,153.0,0.17,0.0,3.0,0.0,3.0,25,6
1,1,3200817575,2022-06-21,2022-06-22,5272903180,ZLF,NaN,DE30,DE30,50034469,...,C,4.25,153.0,0.17,0.0,4.0,0.0,4.0,25,6
2,2,3200817575,2022-06-21,2022-06-22,5272903180,ZLF,NaN,DE30,DE30,50034469,...,C,2.40,84.0,0.40,0.0,1.0,0.0,1.0,25,6
3,3,3200817575,2022-06-21,2022-06-22,5272903180,ZLF,NaN,DE30,DE30,50034469,...,C,2.40,84.0,0.40,0.0,2.0,0.0,2.0,25,6
4,4,3200817575,2022-06-21,2022-06-22,5272903180,ZLF,NaN,DE30,DE30,50034469,...,C,5.40,124.2,0.18,0.0,1.0,0.0,1.0,25,6


In [ ]:
# count unique SKU per day


In [3]:
# group by B and SKU
df_grouped = df.groupby(['B', 'SKU']).sum()
df_grouped.head()

/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_3896/1368400431.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = df.groupby(['B', 'SKU']).sum()


Unnamed: 0            A            D    F          I  \
B          SKU                                                              
2022-04-04 10129020       31820  19211968329  31607968472  0.0  300209348   
           10129077        5610   3201995877   5267990155  0.0   50035357   
           10129175       26172  16009971962  26339989129  0.0  250174240   
           10129201       16240   9605986944  15804068602  0.0  150104218   
           10130014       11173   9605949094  15804150022  0.0  150166567   

                         O    CS    PAL   OUT  Picks PAL  Picks CS  Picks OUT  \
B          SKU                                                                  
2022-04-04 10129020  36.54  32.4  745.2  1.08        0.0       6.0       23.0   
           10129077   5.40   5.4  124.2  0.18        0.0       1.0        0.0   
           10129175  29.88  27.0  621.0  0.90        0.0       5.0       16.0   
           10129201  16.20  16.2  372.6  0.54        0.0       3.0        0.0   
           10130014   6.80  15.0  630.0  0.60        0.0       1.0        9.0   

                     Picks Gesamt  Kalender Woche  Monat  
B          SKU                                            
2022-04-04 10129020          29.0              84     24  
           10129077           1.0              14      4  
           10129175          21.0              70     20  
           10129201           3.0              42     12  
           10130014          10.0              42     12